<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://files.oproject.org/tmvalogo.png" height="50%" width="50%">

# Regression Example

## Declare Factory
Initiate the TMVA library, get the data sample from github, and create a factory to do the regression.

In [1]:
// need to load libblas for a problem in current ROOT master version, 6.13.03 (code = 396547)
if (gROOT->GetVersionCode() >= 396547) gSystem->Load("libblas.so");

In [2]:
TMVA::Tools::Instance();

auto inputFile = TFile::Open("inputdata_regression.root");
auto outputFile = TFile::Open("TMVA_RegressionOutput.root", "RECREATE");

TMVA::Factory factory("TMVARegression", outputFile,
                      "!V:!Silent:Color:DrawProgressBar:AnalysisType=Regression" ); 

## Declare DataLoader
Define the features and the target for the regression.

In [3]:
TMVA::DataLoader loader("dataset"); 

// Add the feature variables, names reference branches in inputFile ttree
loader.AddVariable("var1");
loader.AddVariable("var2");
loader.AddVariable("var3");
loader.AddVariable("var4");
loader.AddVariable("var5 := var1-var3"); // create new features
loader.AddVariable("var6 := var1+var2");

loader.AddTarget( "target := var2+var3" ); // define the target for the regression


## Setup Dataset
Link dataloader to dataset.

In [4]:
TTree *tree;
inputFile->GetObject("Sig", tree);

TCut mycuts = ""; // e.g. TCut mycuts = "abs(var1)<0.5";

loader.AddRegressionTree(tree, 1.0);   // link the TTree to the loader, weight for each event  = 1
loader.PrepareTrainingAndTestTree(mycuts,
                                   "nTrain_Regression=1000:nTest_Regression=1000:SplitMode=Random:NormMode=NumEvents:!V" );

DataSetInfo              : [dataset] : Added class "Regression"
                         : Add Tree Sig of type Regression with 6000 events
                         : Dataset[dataset] : Class index : 0  name : Regression


# Book The Regression Method

Book the method for regression. Here we choose the Boosted Decision Tree model. You have to use gradient boosted trees for regression, hence the BDTG and BoostType=Grad. 

Define the hyperparameters: ntrees, boosttype, shrinkage, and the depth. Also define the loss function you want to use: 'AbsoluteDeviation', 'Huber', or 'LeastSquares'. nCuts determines how finely to look at each feature. Larger values take more time, but you may get more accurate results.

In [5]:
// Boosted Decision Trees 
factory.BookMethod(&loader,TMVA::Types::kBDT, "BDTG",
                   TString("!H:!V:NTrees=64::BoostType=Grad:Shrinkage=0.3:nCuts=20:MaxDepth=4:")+
                   TString("RegressionLossFunctionBDTG=AbsoluteDeviation"));

Factory                  : Booking method: BDTG
                         : 
                         : the option *InverseBoostNegWeights* does not exist for BoostType=Grad --> change
                         : to new default for GradBoost *Pray*
DataSetFactory           : [dataset] : Number of events in input trees
                         : 
                         : Number of training and testing events
                         : ---------------------------------------------------------------------------
                         : Regression -- training events            : 1000
                         : Regression -- testing events             : 1000
                         : Regression -- training and testing events: 2000
                         : 
DataSetInfo              : Correlation matrix (Regression):
                         : --------------------------------------------------------------
                         :               var1    var2    var3    var4 var1-var3 var

# Train Method

In [6]:
factory.TrainAllMethods();

Factory                  : Train all methods
Factory                  : [dataset] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'var1' <---> Output : variable 'var1'
                         : Input : variable 'var2' <---> Output : variable 'var2'
                         : Input : variable 'var3' <---> Output : variable 'var3'
                         : Input : variable 'var4' <---> Output : variable 'var4'
                         : Input : variable 'var5' <---> Output : variable 'var5'
                         : Input : variable 'var6' <---> Output : variable 'var6'
TFHandler_Factory        : Variable        Mean        RMS   [        Min        Max ]
                         : -----------------------------------------------------------
                         :     var1:    0.23401    0.98660   [    -3.3494     3.0772 ]
              

1%, time left: unknown
7%, time left: 0 sec
14%, time left: 0 sec
20%, time left: 0 sec
26%, time left: 0 sec
32%, time left: 0 sec
39%, time left: 0 sec
45%, time left: 0 sec
51%, time left: 0 sec
57%, time left: 0 sec
64%, time left: 0 sec
70%, time left: 0 sec
76%, time left: 0 sec
82%, time left: 0 sec
89%, time left: 0 sec
95%, time left: 0 sec


                         : Elapsed time for training with 1000 events: 0.0722 sec         
                         : Dataset[dataset] : Create results for training
                         : Dataset[dataset] : Evaluation of BDTG on training sample
                         : Dataset[dataset] : Elapsed time for evaluation of 1000 events: 0.00936 sec       
                         : Create variable histograms
                         : Create regression target histograms
                         : Create regression average deviation
                         : Results created
                         : Creating xml weight file: dataset/weights/TMVARegression_BDTG.weights.xml
Factory                  : Training finished
                         : 
Factory                  : === Destroy and recreate all methods via weight files for testing ===
                         : 


0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
18%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
37%, time left: 0 sec
43%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
62%, time left: 0 sec
68%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
93%, time left: 0 sec


# Test and Evaluate the Model

In [7]:
factory.TestAllMethods();
factory.EvaluateAllMethods();    

Factory                  : Test all methods
Factory                  : Test method: BDTG for Regression performance
                         : 
                         : Dataset[dataset] : Create results for testing
                         : Dataset[dataset] : Evaluation of BDTG on testing sample
                         : Dataset[dataset] : Elapsed time for evaluation of 1000 events: 0.0171 sec       
                         : Create variable histograms
                         : Create regression target histograms
                         : Create regression average deviation
                         : Results created
Factory                  : Evaluate all methods
                         : Evaluate regression method: BDTG
TFHandler_BDTG           : Variable        Mean        RMS   [        Min        Max ]
                         : -----------------------------------------------------------
                         :     var1:    0.18637     1.0183   [    -3.3780     3.2875 ]


0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
18%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
37%, time left: 0 sec
43%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
62%, time left: 0 sec
68%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
93%, time left: 0 sec


                         : 
                         : Evaluation results ranked by smallest RMS on test sample:
                         : ("Bias" quotes the mean deviation of the regression from true target.
                         :  "MutInf" is the "Mutual Information" between regression and target.
                         :  Indicated by "_T" are the corresponding "truncated" quantities ob-
                         :  tained when removing events deviating more than 2sigma from average.)
                         : --------------------------------------------------------------------------------------------------
                         : --------------------------------------------------------------------------------------------------
                         : dataset              BDTG           :  0.00749  0.00412    0.171    0.136  |  2.435  2.454
                         : --------------------------------------------------------------------------------------------------
     

## Gather and Plot the Results
Let's plot the residuals for the BDTG predictions. First, close the output file so that it saves to disk and we can open it without issue. Then get the results on the test set. Finally, plot the residuals.

In [8]:
%jsroot on
outputFile->Close();
auto resultsFile = TFile::Open("TMVA_RegressionOutput.root");
auto resultsTree = resultsFile->Get("dataset/TestTree"); 
TCanvas c;
resultsTree->Draw("BDTG-target"); // BDTG is the predicted value, target is the true value
c.Draw();